# Attention Vizualizer

Howdy! Regardless of what shows up below, click Kernel --> Restart and Run All to get started.

Mouse over each Glimpse to view the post-windowing input to the network. It might be kind of slow, since it's a network round trip, so be patient. Enjoy!

In [1]:
from viz import *

Widget Javascript not detected.  It may not be installed or enabled properly.


W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='37f5d8c6-6001-4b3a-9c01-7f2bdda0a24f', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='4fb743a0-9f5f-47f1-9ea0-b49a8694a9fc', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='7643c4fc-3662-4644-ab1e-9b7f7017431e', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='a518e45d-2926-41d0-9150-b5a4370195f2', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='f968a963-7f82-4824-864c-509fd9af5a3a', ...)


INFO:tensorflow:Restoring parameters from model_runs/lined_up/classifymodel_10000.ckpt


In [2]:
from analysis import classify_imgs2

In [8]:
# import numpy as np
# import scipy.special

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf
import scipy

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show


clear_output()
b2 = Button(description="Click to Start", icon="arrow", width=400)

dropdown2 = Dropdown(options=['0', '1000', '2000', '3000', '4000', '5000', '10000', '20000', '30000', '40000', '50000',
                             '60000', '70000', '80000', '90000', '100000', '110000', '120000', '130000', '140000', '150000',
                             '160000', '170000', '180000', '190000', '200000', '250000', '300000', '400000', '500000',
                             '600000', '700000', '800000', '900000', '910000', '920000', '1000000', '1100000', '1200000', '1300000', '1400000', '1500000'],
                    value='10000', 
                    description='Iteration:'
)

data = None
    
    
def get_pdf(sigma, x, mu):
    """Calculate the pdf."""
    
#     pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
    pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    return pdf


def get_cdf(sigma, x, mu, a):
    """Calculate the cdf."""
    
#     cdf = (1 + erf(a*x / sqrt(2))) / 2 
    cdf = (1 + scipy.special.erf((a*x - mu) / np.sqrt(2*sigma**2))) / 2
    return cdf


def get_p(pdf, cdf):
    """Calculate p to create skew curve."""
    
    p = 2 / pdf * cdf / 100000
    return p

    
def curve(sigma, x, mu, plot, label="", color="gray"):
    """Add normal distribution curve to plot."""
    
    pdf = get_pdf(sigma, x, mu)
    plot.line(x, pdf, line_color=color, line_width=8, alpha=1, legend=label)
    
    
def skew_curve(sigma, x, mu, a, plot):
    """Add a skewed curve to the plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))

    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(9-x, p / f(max_x), line_color="blue", line_width=8, alpha=0.4)
    
    
def combined_curve(sigma, x, mu, a, plot):
    """Add curve combining skew and normal distribution to plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))
    
    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    def f2(x): return (1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)) + p / f(max_x))
    max_x2 = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(x, (((1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)))
            + p / f(max_x))/ f2(max_x2)), line_color="purple", line_width=2, alpha=1)
    

def update_curves():
    clear_output()
    global data
    num_imgs = 10000
    print("number of images: %d" % num_imgs)
    imgs_data = classify_imgs2(int(dropdown2.value), True, num_imgs)
    
#     num_blobs = randint(0, 9)
    max_blobs = 9
    min_blobs = 0
    
    curves = list()
    dark = "#111111"
    light = "#DDDDDD"
    p2 = figure(title="Blob Number Classification Probabilities Distributions", y_range=(0, 1), tools="save", background_fill_color=dark)
    p2.border_fill_color = dark
    p2.title.text_color = light
    p2.xaxis.axis_label_text_color = light
    p2.yaxis.axis_label_text_color = light
    p2.xaxis.axis_line_color = light
    p2.yaxis.axis_line_color = light
    p2.xaxis.major_label_text_color = light
    p2.yaxis.major_label_text_color = light
    p2.xaxis.major_tick_line_color = light
    p2.xaxis.minor_tick_line_color = light
    p2.yaxis.major_tick_line_color = light
    p2.yaxis.minor_tick_line_color = light

    for num_blobs in range(min_blobs, max_blobs + 1):
        print("num_blobs: ", num_blobs)
        

        p1 = figure(title="Blob Number Classification Probabilities Distribution for %d Blobs" % num_blobs, y_range=(0, 1), tools="save",
                    background_fill_color=dark)
        p1.border_fill_color = dark
        p1.title.text_color = light
        p1.xaxis.axis_label_text_color = light
        p1.yaxis.axis_label_text_color = light
        p1.xaxis.axis_line_color = light
        p1.yaxis.axis_line_color = light
        p1.xaxis.major_label_text_color = light
        p1.yaxis.major_label_text_color = light
        p1.xaxis.major_tick_line_color = light
        p1.xaxis.minor_tick_line_color = light
        p1.yaxis.major_tick_line_color = light
        p1.yaxis.minor_tick_line_color = light



        m = 0.1
        z_size = 10

        new_hist = np.zeros(z_size)
        choice_hist = np.zeros(z_size)
        value_counts = np.zeros(z_size)
        values_sum = 0
        sqr_sum = 0
        num_imgs_with_num_blobs = 0

        for idx, data in enumerate(imgs_data):

            if data["label"][num_blobs] == 1: # data is for an image with num_blobs blobs
                num_imgs_with_num_blobs += 1
                
                max_glimpse = 2
                min_glimpse = 0
                glimpses = 5#max_glimpse - min_glimpse + 1
                
                for i in [0, 1, 2, 3, 4]:#range(min_glimpse, max_glimpse + 1):
                    # Histogram of softmaxes
                    new_hist += data["classifications"][i][0] / glimpses
                    
                    # Histogram of choices
                    choice = np.argmax(data["classifications"][i][0])
                    choice_list = [0] * z_size
                    choice_list[choice] = 1 / glimpses
                    choice_hist += choice_list
                    
#                 glimpse = 1
        
#                 new_hist += data["classifications"][glimpse][0]
        
#                 choice = np.argmax(data["classifications"][glimpse][0])
#                 choice_list = [0] * z_size
#                 choice_list[choice] = 1
#                 choice_hist += choice_list
                
        print("num_imgs_with_num_blobs: ", num_imgs_with_num_blobs)
        
        new_hist = new_hist / num_imgs_with_num_blobs
        print("new_hist: ", new_hist)
        choice_hist = choice_hist / num_imgs_with_num_blobs

        x = np.linspace(-2, 11.0, 1000)
        source = ColumnDataSource(data=dict(color=["red"] * z_size, top=new_hist, bottom=np.zeros(z_size), left=np.arange(z_size) + m - 0.55, right=np.arange(1, z_size + 1) - m - 0.55))
        source2 = ColumnDataSource(data=dict(color=["yellow"] * z_size, top=choice_hist, bottom=np.zeros(z_size), left=np.arange(z_size) + m - 0.45, right=np.arange(1, z_size + 1) - m - 0.45))
        source.data["color"][num_blobs] = "lime"
        p1.quad('left', 'right', 'top', 'bottom', source=source, color="color", alpha=1)
        p1.quad('left', 'right', 'top', 'bottom', source=source2, color="color", alpha=0.5)


        # FORMAT PLOT ##############################

        p1.xaxis.axis_label = 'Number of Blobs'
        p1.yaxis.axis_label = 'Classification Probability'
        p1.xaxis[0].ticker=FixedTicker(ticks=np.arange(z_size))
        
        
        # PLOT CURVES #############################
        
        # Find the mean
        for j in range(z_size):
            values_sum += j * new_hist[j]
        mu = values_sum
        print("mu: ", mu)

        # Find the standard deviation
        for k in range(z_size):
            sqr_sum += new_hist[k] * ((k - mu) ** 2)
        sigma = np.sqrt(sqr_sum)
        
        curves.append((sigma * 2, x, mu, p2, str(num_blobs),
                       "#" + str(randint(2, 9)) + str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))))
        
        curve(sigma, x, mu, p1) # plot gaussian curve
#         a = np.argmax(new_hist)
#         skew_curve(sigma, x, mu, a, p1)
#         combined_curve(sigma, x, mu, a, p1)

        show(gridplot(p1, ncols=2, plot_width=500, plot_height=400, toolbar_location=None))
    
    for stats in curves:
        curve(*stats)
    show(p2)
    

def on_click2(b2, new_image=True):
    """Load new random image after button is clicked."""
    
    b2.description = "Loading..."
    update_curves()
    b2.description = "Next (Random) Image"

b2.on_click(on_click2)


def on_change2(change):
    """Change the iteration number to new dropdown selection."""
    
    if change['type'] == 'change' and change['name'] == 'value':
        on_click(b2, new_image=False)
        

dropdown2.observe(on_change2)
display(HBox([b2, dropdown2]))

number of images: 10000
INFO:tensorflow:Restoring parameters from model_runs/lined_up/classifymodel_10000.ckpt
num_blobs:  0
num_imgs_with_num_blobs:  1000
new_hist:  [  9.76105407e-01   1.12565747e-02   7.16293330e-03   3.47342355e-03
   4.40776472e-04   4.67245821e-04   1.10852296e-04   9.36831779e-05
   3.87066764e-04   5.02025596e-04]
mu:  0.0490377074069


num_blobs:  1
num_imgs_with_num_blobs:  1000
new_hist:  [  1.64679086e-02   9.57488590e-01   2.36276788e-02   1.07302381e-03
   2.10373392e-04   2.85793114e-04   2.07080942e-05   1.58595662e-05
   3.27684138e-04   4.82403252e-04]
mu:  1.0174318464


num_blobs:  2
num_imgs_with_num_blobs:  1000
new_hist:  [  3.42825060e-04   4.55863182e-02   9.27259555e-01   2.45639930e-02
   6.00061982e-04   4.21831954e-04   5.65075037e-06   2.27740211e-06
   6.28750511e-04   5.88744452e-04]
mu:  1.98868536593


num_blobs:  3
num_imgs_with_num_blobs:  1000
new_hist:  [  8.44741759e-06   5.20190729e-05   6.42575399e-02   9.01420135e-01
   3.27352985e-02   1.22718398e-03   4.06180181e-06   9.77776860e-07
   2.28469986e-04   6.58685123e-05]
mu:  2.97235640977


num_blobs:  4
num_imgs_with_num_blobs:  1000
new_hist:  [  1.99717294e-07   4.73127573e-08   3.81769619e-05   7.25851511e-02
   9.09516409e-01   1.77066058e-02   4.05125040e-05   1.97716459e-06
   1.09525736e-04   1.39831164e-06]
mu:  3.94557622688


num_blobs:  5
num_imgs_with_num_blobs:  1000
new_hist:  [  1.59798081e-09   2.60950581e-12   9.39887288e-10   1.64648707e-04
   8.36666492e-02   8.66606606e-01   4.94058725e-02   9.69594199e-05
   5.92586978e-05   4.77135467e-09]
mu:  4.96578164035


num_blobs:  6
num_imgs_with_num_blobs:  1000
new_hist:  [  2.42266581e-11   2.29045684e-16   1.93402591e-16   3.96169393e-07
   6.66725794e-05   4.71127964e-02   9.29371558e-01   2.29824515e-02
   4.66125089e-04   4.21740369e-12]
mu:  5.97666737252


num_blobs:  7
num_imgs_with_num_blobs:  1000
new_hist:  [  2.07753515e-14   9.22021169e-24   4.61255583e-26   8.58695848e-12
   5.78327085e-10   1.82777020e-05   9.74739882e-02   7.88948201e-01
   1.13559534e-01   3.53081433e-17]
mu:  7.01604899744


num_blobs:  8
num_imgs_with_num_blobs:  1000
new_hist:  [  3.71303613e-17   1.92117542e-29   1.91383717e-33   2.13446460e-16
   2.95734523e-16   1.42171319e-09   1.37990771e-04   7.79805032e-02
   9.21881507e-01   6.15236730e-22]
mu:  7.92174353098


num_blobs:  9
num_imgs_with_num_blobs:  1000
new_hist:  [  2.83590167e-20   1.48202198e-33   1.91667631e-40   3.14064106e-21
   1.25035427e-22   3.04120980e-14   6.62048781e-08   6.47564139e-05
   9.99935191e-01   4.63719946e-27]
mu:  7.99993521719


In [4]:
edges # the left and right edges of the bars

NameError: name 'edges' is not defined

In [ ]:
hist # the heights of the bars

In [ ]:
len(data["classifications"]) # glimpses

In [ ]:
len(data["classifications"][0]) # machine and human (2)

In [ ]:
len(data["classifications"][0][0])

In [ ]:
3 ** 2

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf

def pdf(x):
    return 1/sqrt(2*pi) * exp(-x**2/2)

def cdf(x):
    return (1 + erf(x/sqrt(2))) / 2

def skew(x,e=0,w=1,a=0):
    t = (x-e) / w
    return 2 / w * pdf(t) * cdf(a*t)
    # You can of course use the scipy.stats.norm versions
    # return 2 * norm.pdf(t) * norm.cdf(a*t)


n = 2**10

e = 1.0 # location
w = 2.0 # scale

x = linspace(-10,10,n) 

for a in range(-3,4):
    p = skew(x,e,w,a)
    plt.plot(x,p)
    plt.show()

show()